In [2]:
from qiskit_ibm_runtime.fake_provider import FakeAuckland
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import Batch
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit.quantum_info import SparsePauliOp


# Initialize the Qiskit IBM Runtime service
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='dd810d4274a18946b6783e5557a9e2a912c07a2dd722153fd10e5dd5fa7512521eb33cad123947c69ff02debf01e4615b67460fbb51fd94cb2f2b6153d5401d9'
)

In [39]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import BackendEstimatorV2

# Initialize IBM Quantum service
backend = service.least_busy(operational=True, simulator=False)

# Define a simple quantum circuit
qc = QuantumCircuit(1)
qc.h(0)

# Transpile with ALAP scheduling
qc_transpiled = transpile(qc, backend, scheduling_method="alap", optimization_level=0)

# Define an observable (Pauli operator)
observable_str = "X" + "I" * (backend.num_qubits - 1)
observable = SparsePauliOp(observable_str)

# Initialize BackendEstimatorV2
estimator = BackendEstimatorV2(backend=backend)
estimator.options.shots = 1000  # Set number of shots

# Run estimation (PUB format: (circuit, observable, parameter_values))
job = estimator.run([(qc_transpiled, observable, [])])  # Pass empty parameter list []
result = job.result()

print(result)


IBMBackendError: 'Support for backend.run() has been removed. Please see our migration guide https://docs.quantum.ibm.com/migration-guides/qiskit-runtime for instructions on how to migrate to the primitives interface.'

In [64]:
from qiskit.transpiler import InstructionDurations
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler.passes import ALAPSchedule
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
from qiskit import generate_preset_pass_manager

# Initialize IBM Quantum service
backend = service.least_busy(operational=True, simulator=False)

# Define quantum circuit
qc = QuantumCircuit(1)
qc.h(0)

# Add delay (5 microseconds)
qc.delay(5, 0, unit="us")  # Ensure correct unit

# Define an observable (Pauli operator)
observable_str = "X"
observable = SparsePauliOp(observable_str)

# 🔹 Step 1: Define gate durations manually (Corrected)
durations = InstructionDurations()
durations.update("h", [0], 100)
durations.update("delay", [0], 5 * backend.configuration().dt)# Convert delay to dt units

# 🔹 Step 2: Apply scheduling with explicitly defined durations
scheduled_qc = ALAPSchedule(durations)(qc)  # Apply ALAP scheduling

# Generate preset pass manager and transpile the scheduled circuit
pm = generate_preset_pass_manager(optimization_level=0, backend=backend)
isa_circuit = pm.run(scheduled_qc)

# Apply layout to observable
isa_observable = observable.apply_layout(isa_circuit.layout)

# Initialize estimator
estimator = Estimator(mode=backend)

# Run estimation
job = estimator.run([(isa_circuit, isa_observable, [])])

print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")
result = job.result()
print(f"  > Expectation value: {result[0].data.evs}")
print(f"  > Metadata: {result[0].metadata}")


TypeError: InstructionDurations.update() takes from 2 to 3 positional arguments but 4 were given

In [48]:
result = job.result()
print(f">>> {result}")
print(f"  > Expectation value: {result[0].data.evs}")
print(f"  > Metadata: {result[0].metadata}")

>>> PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.ndarray(<shape=(), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(), dtype=float64>)), metadata={'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})], metadata={'dynamical_decoupling': {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})
  > Expectation value: 0.975597609561753
  > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


In [38]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import BackendEstimatorV2

# Initialize IBM Quantum service
backend = service.least_busy(operational=True, simulator=False)

# Define a simple quantum circuit
qc = QuantumCircuit(1)
qc.h(0)

# Transpile with ALAP scheduling
qc_transpiled = transpile(qc, backend, scheduling_method="alap", optimization_level=0)

# Define an observable (Pauli operator)
observable_str = "X" + "I" * (backend.num_qubits - 1)
observable = SparsePauliOp(observable_str)

# Initialize BackendEstimatorV2
estimator = BackendEstimatorV2(backend=backend)
estimator.options.shots = 1000  # Set number of shots

# Run estimation using the correct argument structure for BackendEstimatorV2
job = estimator.run(qc_transpiled, observable)  # Pass empty parameter list [[]]
result = job.result()

print(result)


TypeError: BackendEstimatorV2.run() got an unexpected keyword argument 'circuits'

In [4]:
from qiskit import transpile

# backend = service.least_busy(operational=True, simulator=False)
backend = FakeAuckland()
qc_transpiled = transpile(qc, backend)  # Makes circuit compatible with target hardware

estimator = Estimator(mode=backend)
job = estimator.run([(qc_transpiled, "X"*27)])

In [ ]:
result = job.result()
print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

In [ ]:
print("3")

In [3]:
from qiskit.circuit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passmanager import PassManager

from qiskit_ibm_runtime.transpiler.passes.scheduling import DynamicCircuitInstructionDurations
from qiskit_ibm_runtime.transpiler.passes.scheduling import ALAPScheduleAnalysis
from qiskit_ibm_runtime.transpiler.passes.scheduling import PadDelay
from qiskit_ibm_runtime.fake_provider import FakeJakartaV2

backend = service.least_busy(operational=True, simulator=False)

# Use this duration class to get appropriate durations for dynamic
# circuit backend scheduling
durations = DynamicCircuitInstructionDurations.from_backend(backend)
# Generate the main Qiskit transpile passes.
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
# Configure the as-late-as-possible scheduling pass
pm.scheduling = PassManager([ALAPScheduleAnalysis(durations), PadDelay(durations)])

qr = QuantumRegister(1)
crz = ClassicalRegister(1, name="crz")
crx = ClassicalRegister(1, name="crx")
result = ClassicalRegister(1, name="result")

qc = QuantumCircuit(qr, crz, crx, result, name="Teleport")

qc.h(qr[0])
qc.h(qr[0])
qc.delay(100, qr[0])

# Transpile.
scheduled_teleport = pm.run(qc)

# scheduled_teleport.draw(output="mpl", style="iqp")

In [4]:
from qiskit.circuit.library import XGate

from qiskit_ibm_runtime.transpiler.passes.scheduling import PadDynamicalDecoupling


dd_sequence = [XGate(), XGate()]

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
pm.scheduling = PassManager(
    [
        ALAPScheduleAnalysis(durations),
        PadDynamicalDecoupling(durations, dd_sequence),
    ]
)

dd_teleport = pm.run(qc)

# dd_teleport.draw(output="mpl", style="iqp")

In [5]:
# Initialize estimator
estimator = Estimator(mode=backend)


observable_str = "X"
observable = SparsePauliOp(observable_str)
isa_observable = observable.apply_layout(qc.layout)

# Run estimation
job = estimator.run([(qc, isa_observable, [])])

print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")
result = job.result()
print(f"  > Expectation value: {result[0].data.evs}")
print(f"  > Metadata: {result[0].metadata}")


IBMInputValueError: 'The instruction h on qubits (0,) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/guides/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/guides/primitives-examples) to see this coupled with operator transformations.'

TranspilerError: 'The input circuit None is not scheduled. Call one of scheduling passes before running the ConstrainedReschedule pass.'

In [7]:

# 1. Build the quantum circuit with a Hadamard and a delay
qc = QuantumCircuit(1, 1)        # 1 qubit, 1 classical bit
qc.h(0)                          # Apply Hadamard on qubit 0
qc.delay(100, 0, unit="dt")      # Insert a delay of 100 dt on qubit 0
qc.measure(0, 0)                 # Measure qubit 0

print("Original circuit:")
print(qc)

# 2. Transpile the circuit to the backend’s native gate set
qc_transpiled = transpile(qc, backend=backend)
print("\nTranspiled circuit:")
print(qc_transpiled)

# 3. Execute the transpiled circuit on the backend
job = backend.run(qc_transpiled, shots=1024)
result = job.result()
print("\nMeasurement counts:", result.get_counts())

Original circuit:
     ┌───┐┌────────────────┐┌─┐
  q: ┤ H ├┤ Delay(100[dt]) ├┤M├
     └───┘└────────────────┘└╥┘
c: 1/════════════════════════╩═
                             0 


TranspilerError: 'The input circuit None is not scheduled. Call one of scheduling passes before running the ConstrainedReschedule pass.'